In [88]:
import os
import sys

import numpy as np
from tqdm import trange
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.convolution import convolve, Gaussian1DKernel
import astropy.units as u
import astropy.coordinates as coord
import matplotlib
import matplotlib.pyplot as plt
from astropy.table import Column
from tqdm import trange
import pandas as pd
import fitsio
from astropy.table import Table, vstack
from astropy import units as u
from astropy.coordinates import SkyCoord
from easyquery import Query, QueryMaker
from scipy.stats import binomtest
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm
from matplotlib.colors import ListedColormap, BoundaryNorm
import h5py
from astropy.cosmology import Planck18

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

rootdir = '/global/u1/v/virajvm/'
sys.path.append(os.path.join(rootdir, 'DESI2_LOWZ/desi_dwarfs/code'))

from desi_lowz_funcs import make_subplots

In [87]:
# clean_cat = Table.read("/pscratch/sd/v/virajvm/catalog_dr1_dwarfs/desi_y1_dwarf_clean_catalog_v4.fits")
# #subselect:

# print(f"Total number = {len(clean_cat)}")

# ## we select the first 10k clean objects in each sample
# clean_lowz = clean_cat[clean_cat["SAMPLE"] == "LOWZ"][:10000]
# clean_elg = clean_cat[clean_cat["SAMPLE"] == "ELG"][:10000]
# clean_bgsb = clean_cat[clean_cat["SAMPLE"] == "BGS_BRIGHT"][:10000]
# clean_bgsf = clean_cat[clean_cat["SAMPLE"] == "BGS_FAINT"][:10000]

# #let us save this as one clean cat
# clean_cat_sub = vstack([ clean_bgsb, clean_bgsf, clean_lowz, clean_elg ])

# clean_cat_sub.write("/pscratch/sd/v/virajvm/catalog_dr1_dwarfs/desi_y1_dwarf_clean_catalog_v4_RUN_W_APER.fits", overwrite=True)


Total number = 379185


In [89]:
dwarf_cat = Table.read("/pscratch/sd/v/virajvm/catalog_dr1_dwarfs/iron_sga_matched_dwarfs.fits")

In [59]:
import glob

all_files = glob.glob("/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/*")

print(len(all_files))

40457


In [72]:
glob.glob("/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627362920307565*")

['/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627362920307565_ra_28.750_dec_-17.632334.fits',
 '/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627362920307565_ra_28.750_dec_-17.632.fits']

In [23]:
clean_cat = Table.read("/pscratch/sd/v/virajvm/catalog_dr1_dwarfs/desi_y1_dwarf_clean_catalog_v4_RUN_W_APER.fits")


In [33]:
clean_cat[clean_cat["RA"] == 61.214696530120904]

TARGETID,SURVEY,PROGRAM,HEALPIX,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,COADD_FIBERSTATUS,RA,DEC,PMRA,PMDEC,REF_EPOCH,FA_TARGET,FA_TYPE,OBJTYPE,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FIBERFLUX_R,MASKBITS,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SCND_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV1_SCND_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV2_SCND_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,PLATE_RA,PLATE_DEC,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_PSF_TO_FIBER_SPECFLUX,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MIN_MJD,MAX_MJD,MEAN_MJD,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,MAIN_NSPEC,MAIN_PRIMARY,SV_NSPEC,SV_PRIMARY,ZCAT_NSPEC,ZCAT_PRIMARY,DESINAME,FLUX_G,FLUX_IVAR_G,MAG_G,MAG_G_ERR,FLUX_R,FLUX_IVAR_R,MAG_R,MAG_R_ERR,FLUX_Z,FLUX_IVAR_Z,MAG_Z,MAG_Z_ERR,FIBERMAG_R,OBJID,SIGMA_G,FRACFLUX_G,RCHISQ_G,SIGMA_GOOD_G,SIGMA_R,FRACFLUX_R,RCHISQ_R,SIGMA_GOOD_R,SIGMA_Z,FRACFLUX_Z,RCHISQ_Z,SIGMA_GOOD_Z,SHAPE_R,SHAPE_R_ERR,MU_R,MU_R_ERR,SERSIC,SERSIC_IVAR,BA,TYPE,PHI,NOBS_G,NOBS_R,NOBS_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,HALPHA_FLUX_CG,HALPHA_FLUXERR_CG,HALPHA_EW_CG,HALPHA_EWERR_CG,MASS_CG,MASSERR_CG,LOGM_SAGA,LOGM_M24,SWEEP,is_south,STARFDIST,STARDIST_DEG,STARMAG,STAR_RADIUS_ARCSEC,STAR_RA,STAR_DEC,IMAGE_SIZE_PIX,SAMPLE,SGA_ID_MATCH,SGA_D26_NORM_DIST,SGA_DIST_DEG,IMAGE_PATH,FILE_PATH
,,,,,,,,,,,,,,,,deg,deg,mas / yr,mas / yr,yr,,,,,,,,,,,mag,nanomaggy,,,,mag,mag,mag,mas,,,,,,,,,,,,,,,,,,,,,deg,deg,,s,,,mm,mm,mm,mm,,deg,arcsec,deg,arcsec,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,nanomaggy,1/nanomaggy^2,nanomaggy,1/nanomaggy^2,nanomaggy,1/nanomaggy^2,nanomaggy,1/nanomaggy^2,nanomaggy,1/nanomaggy^2,nanomaggy,1/nanomaggy^2,nanomaggy,,nanomaggy,,,,nanomaggy,,,,nanomaggy,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,bytes7,bytes6,int32,int32,float64,float64,int64,float64,float64[10],int64,bytes6,bytes20,int64,float64,int32,float64,float64,float32,float32,float32,int64,uint8,bytes3,float64,int32,int16,bytes8,int32,int32,bytes4,float32,float32,int16,int64,bytes2,float32,float32,float32,float32,bytes1,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int16,float32,int16,int16,float32,float32,float32,float32,float32,float64,float32,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,bool,int16,bool,int16,bool,bytes22,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes3,float32,int16,int16,int16,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,bytes29,int64,float32,float32,float32,float32,float32,float32,int64,bytes10,int64,float64,float64,bytes121,bytes124
39627082589803927,sv1,bright,34725,34725,0.059369732681056685,1.7928738202246708e-05,0,7972.17561298795,1.56050197229699 .. -28.313115785612823,7927,GALAXY,--,10,61.03197787515819,0,61.214

In [50]:
img_path = clean_cat[clean_cat["TARGETID"] == 39627374345585658]
img_path

KeyError: 'IMAGE_PATH'

In [58]:
glob.glob("/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627369299841306*")


['/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627369299841306_ra_67.174_dec_-17.455.fits',
 '/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627369299841306_ra_67.174_dec_-17.454553.fits']

In [56]:
os.path.exists(img_path)

False

In [64]:

img_good_path = "/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts"

from tqdm.notebook import tqdm

## construct the dictionary
def preload_file_dict(directory):
    """
    Load all filenames in a directory and map tgid -> filepath.
    Assumes filenames contain 'tgid_1234567890' or similar.
    """
    file_paths = glob.glob(os.path.join(directory, "image_tgid_*ra*dec*"))
    file_dict = {}
    for path in tqdm(file_paths):
        base = os.path.basename(path)
        if "tgid_" in base:
            try:
                tgid = int(base.split("tgid_")[1].split("_")[0])
                file_dict[tgid] = path
            except:
                continue
    return file_dict


In [65]:
good_img_dict = preload_file_dict(img_good_path)

  0%|          | 0/40457 [00:00<?, ?it/s]

In [77]:
good_img_dict[39627380200833519]

'/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627380200833519_ra_26.399_dec_-16.897450.fits'

In [81]:
glob.glob("/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627351511794521*")

['/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627351511794521_ra_34.451_dec_-18.210.fits',
 '/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts/image_tgid_39627351511794521_ra_34.451_dec_-18.210139.fits']

In [83]:
import os
import shutil
import glob

src_dir = "/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts"
dst_dir = "/pscratch/sd/v/virajvm/redo_photometry_plots/all_good_cutouts_nodups"

os.makedirs(dst_dir, exist_ok=True)

all_files_loop = glob.glob(os.path.join(src_dir, "*.fits"))

for fname in tqdm(all_files_loop):
    basename = os.path.basename(fname)

    # Example: image_tgid_39627380200833519_ra_26.399_dec_-16.897450.fits
    parts = basename.replace(".fits", "").split("_")
    tgid = parts[2]
    ra   = float(parts[4])
    dec  = float(parts[6])

    new_name = f"image_tgid_{tgid}_ra_{ra:.3f}_dec_{dec:.3f}.fits"
    dst_path = os.path.join(dst_dir, new_name)

    shutil.copy2(fname, dst_path)   # overwrites if already exists

print("Done.")


  0%|          | 0/40648 [00:00<?, ?it/s]

Done.


In [8]:
# DESI targeting masks - 
from desitarget.sv1 import sv1_targetmask    # For SV1
from desitarget.sv2 import sv2_targetmask    # For SV2
from desitarget.sv3 import sv3_targetmask    # For SV3
from desitarget import targetmask



In [2]:
zpix_iron = Table.read("/global/cfs/cdirs/desi/spectro/redux/iron/zcatalog/v1/zall-pix-iron.fits")

#we remove these columns as we will be adding columns from the zpix_trac later and so hope to avoid confusion
cols_to_remove = [
"FLUX_G", "FLUX_R", "FLUX_Z", "FLUX_W1", "FLUX_W2",
"FLUX_IVAR_G", "FLUX_IVAR_R", "FLUX_IVAR_Z", "FLUX_IVAR_W1", "FLUX_IVAR_W2",
"FIBERFLUX_G", "FIBERFLUX_Z",
"FIBERTOTFLUX_G", "FIBERTOTFLUX_R", "FIBERTOTFLUX_Z",
"SERSIC", "SHAPE_R", "SHAPE_E1", "SHAPE_E2"]


zpix_iron.remove_columns(cols_to_remove)



##rename columns
zpix_iron.rename_column('TARGET_RA', 'RA')
zpix_iron.rename_column('TARGET_DEC', 'DEC')


In [3]:

##apply a basic cut of spectype = "GALAXY", z > 0.001 and ZCAT_PRIMARY = 1, and maximum redshift
basic_cleaning = (zpix_iron["SPECTYPE"] == "GALAXY") & (zpix_iron["ZCAT_PRIMARY"] == 1) & (zpix_iron["COADD_FIBERSTATUS"] == 0) & (zpix_iron["Z"] < 0.5) & (zpix_iron["Z"] > 0.001)

#we apply a cut on redshift, because we are using redshifts to get distances and we cannot get accurate stellar masses anyways if z<0.001

print(f"Fraction remaining after basic cleaning cut = { np.sum(basic_cleaning)/len(zpix_iron) }")

zpix_iron = zpix_iron[basic_cleaning]

#select for unique targets
_, uni_tgids = np.unique( zpix_iron["TARGETID"].data, return_index=True )
zpix_iron = zpix_iron[uni_tgids]



Fraction remaining after basic cleaning cut = 0.27931008001382396


In [ ]:

gal_types = ["BGS_BRIGHT", "BGS_FAINT", "ELG","LOWZ"]


#looping over all the sub-samples!
#gal_mask is the boolean array to select the galaxies
for i,gal_type in enumerate(gal_types):

    if gal_type == "LOWZ":
        zpix_iron = zpix_iron[zpix_iron["PROGRAM"] == "dark"]

        #get the lowz survey mask
        lowz_main_mask = (zpix_iron["SCND_TARGET"] == 2**15) | (zpix_iron["SCND_TARGET"] == 2**16) | (zpix_iron["SCND_TARGET"] == 2**17)
        lowz_sv_mask = np.zeros(len(zpix)).astype(bool)
        for svi in range(1,4):
            svi_mask = (zpix[f"SV{svi}_SCND_TARGET"] == 2**15) | (zpix[f"SV{svi}_SCND_TARGET"] == 2**16) | (zpix[f"SV{svi}_SCND_TARGET"] == 2**17)
            lowz_sv_mask |= svi_mask
    
        gal_mask = lowz_main_mask | lowz_sv_mask
            
        
    if gal_type == "BGS_BRIGHT":
        iron_bgs_tgid = zpix_iron["BGS_TARGET"]
        bgsb_main_mask = ( (iron_bgs_tgid & bgs_mask["BGS_BRIGHT"]) != 0 )
        bgsb_sv_mask = get_sv_bgs_mask(zpix_iron, bgs_class = "BGS_BRIGHT")

        print(f"Number of BGS Bright objects in Main survey = {np.sum(bgsb_main_mask)}")
        print(f"Number of BGS Bright objects in SV survey = {np.sum(bgsb_sv_mask)}")
        
        gal_mask = bgsb_main_mask | bgsb_sv_mask
        
    if gal_type == "BGS_FAINT":
        iron_bgs_tgid = zpix_iron["BGS_TARGET"]
        
        bgsf_main_mask = ( (iron_bgs_tgid & bgs_mask["BGS_FAINT"]) != 0 )

        bgsf_sv_mask = get_sv_bgs_mask(zpix_iron, bgs_class = "BGS_FAINT")

        print(f"Number of BGS Faint objects in Main survey = {np.sum(bgsf_main_mask)}")
        print(f"Number of BGS Faint objects in SV survey = {np.sum(bgsf_sv_mask)}")
        
        gal_mask = bgsf_main_mask | bgsf_sv_mask

    if gal_type == "ELG":
        desi_tgt = zpix_iron["DESI_TARGET"]
        elg_main_mask = (desi_tgt & desi_mask["ELG"]) != 0
        elg_sv_mask = get_sv_elg_mask(zpix_iron)
        gal_mask = elg_main_mask | elg_sv_mask




The weird issue of trying to select with fastspec catalog

In [9]:
iron_vac = fits.open("/global/cfs/cdirs/desi/public/dr1/vac/dr1/fastspecfit/iron/v2.1/catalogs/fastspec-iron.fits")
vac_data_f = iron_vac[2].data


In [10]:
## METHOD 1

sel = (vac_data_f["SV1_BGS_TARGET"] & sv1_targetmask.bgs_mask["BGS_BRIGHT"] != 0)

print(len(vac_data_f[sel]))

16692760


In [11]:
## METHOD 2

sv1_desi_tgt =vac_data_f['SV1_DESI_TARGET']
## first select for SV1 BGS ANY objects
sv1_desi_mask = sv1_targetmask.desi_mask
sv1_bgs = (sv1_desi_tgt & sv1_desi_mask['BGS_ANY'] != 0)
vac_data_sv1 = vac_data_f[sv1_bgs]

## then select by bgs bright
sv1_bgs_bright = (vac_data_sv1 ["SV1_BGS_TARGET"] & sv1_targetmask.bgs_mask['BGS_BRIGHT'] != 0)

print(len(vac_data_sv1[sv1_bgs_bright ]))



55283
